In [18]:
import pandas as pd 
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score
from sklearn.naive_bayes import MultinomialNB
import pickle
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils import resample
pd.set_option('display.max_columns', 50)

In [5]:
df = pd.read_csv('train.csv')

In [4]:
df.head(10)

benefits_review  \
0  slowed the progression of left ventricular dys...   
1  Although this type of birth control has more c...   
2  I was used to having cramps so badly that they...   
3  The acid reflux went away for a few months aft...   
4  I think that the Lyrica was starting to help w...   
5  after taking propecia for over a year, startin...   
6  My mood has noticably improved, I have more en...   
7  Although this drug was originally prescribed f...   
8  This simply just works fast and without any of...   
9  none - did nothing to help allergies. i just h...   

                                 side_effects_review  \
0  cough, hypotension , proteinuria, impotence , ...   
1  Heavy Cycle, Cramps, Hot Flashes, Fatigue, Lon...   
2         Heavier bleeding and clotting than normal.   
3  Constipation, dry mouth and some mild dizzines...   
4  I felt extremely drugged and dopey.  Could not...   
5  Low sex drive, before i started taking this dr...   
6  a few experiences of nausiea, heavy moodswings...   
7                                              None.   
8  I really don't have any side effects other tha...   
9  I had some horrifying mental and physical side...   

                                     comments_review  rating  
0  monitor blood pressure , weight and asses for ...       4  
1  I Hate This Birth Control, I Would Not Suggest...       1  
2  I took 2 pills at the onset of my menstrual cr...      10  
3  I was given Prilosec prescription at a dose of...       3  
4                                          See above       2  
5                                one pill once daily       1  
6  I had began taking 20mg of Vyvanse for three m...       9  
7     One a day, taken about an hour before bedtime.      10  
8  I first started taking this at 3 times per day...      10  
9                       took one 10 mg pill nightly.       1

In [6]:
df1 = df.copy()

In [7]:
def combine(df1):
    df1['review'] = df1['benefits_review'] + " " + df1['side_effects_review'] + " " + df1['comments_review']
    df1['sentiment'] = df1.rating.apply(lambda x: 'positive' if x >= 6 else 'negative')
    df1 = df1.drop(['rating', 'benefits_review', 'side_effects_review', 'comments_review'], axis=1)
    return df1

In [8]:
df1 = combine(df1)

In [10]:
df1.sentiment.value_counts()

positive    2281
negative     816
Name: sentiment, dtype: int64

In [20]:
df1_pos = df1[df1.sentiment=='positive']
df1_neg = df1[df1.sentiment=='negative']

In [21]:
df_pos_downsampled = resample(df1_pos, 
                             replace=False,    
                             n_samples=len(df1_neg),    
                             random_state=42)  

In [22]:
df1_balanced = pd.concat([df_pos_downsampled, df1_neg])

In [23]:
df1_balanced.sentiment.value_counts()

positive    816
negative    816
Name: sentiment, dtype: int64

In [24]:
X = df1_balanced.drop('sentiment', axis = 1)
y = df1_balanced.sentiment

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [26]:
X_train_docs = [doc for doc in X_train.review]

In [27]:
pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', MultinomialNB())
])

In [28]:
pipeline.fit(X_train_docs, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('clf', MultinomialNB())])

In [29]:
cross_val_score(pipeline, X_train_docs, y_train, cv = 5).mean()

0.7346280548532904

In [30]:
df_test = pd.read_csv('test.csv')

In [31]:
df1_test = df_test.copy()

In [32]:
df1_test = combine(df1_test)

In [33]:
predicted_test = pipeline.predict([doc for doc in df1_test.review])

In [34]:
accuracy_score(df1_test.sentiment, predicted_test)

0.7596525096525096

In [35]:
f1_score(df1_test.sentiment, predicted_test, pos_label= 'positive' )

0.8145941921072227

In [37]:
with open('pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

In [41]:
pred_df = pd.DataFrame({'Predicted' : pipeline.predict([doc for doc in df1_test.review]), 
                        'Actual': df1_test.sentiment}) 

In [42]:
pred_df

Predicted    Actual
0     negative  positive
1     positive  positive
2     negative  negative
3     positive  positive
4     negative  positive
...        ...       ...
1031  positive  positive
1032  positive  positive
1033  negative  positive
1034  positive  positive
1035  positive  positive

[1036 rows x 2 columns]